In [1]:
#https://medium.com/huggingface/introducing-fastbert-a-simple-deep-learning-library-for-bert-models-89ff763ad384
import torch
import apex

from pytorch_pretrained_bert.tokenization import BertTokenizer

from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy

In [ ]:
# generate train, val, and labels files
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')

In [8]:
DATA_PATH = '../input/'     # path for data files (train and val)
LABEL_PATH = '../labels/'  # path for labels file
MODEL_PATH='../models/'    # path for model artifacts to be stored
LOG_PATH='../logs/'       # path for log files to be stored

# location for the pretrained BERT models
BERT_PRETRAINED_PATH = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'

args = {
    "max_seq_length": 512,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [9]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, 
                                          do_lower_case=args['do_lower_case'])

In [10]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer, 
                          train_file='train.csv', val_file='val.csv', label_file='labels.csv',
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=False)